In [1]:
from modules.base import *

#### 1. 요약 사용하기

In [2]:
LIMIT_LENGTH = 10

In [3]:
class State(MessagesState):
    summary: str

@trace_function(enable_print=False)
def conversation(state: State, 
                 config: RunnableConfig):
    
    summary = state.get("summary", "")
    if summary:
        system_message = f"다음은 지금까지의 대화 요약입니다. {summary}\n\n"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = llm.invoke(messages, config)
    return {"messages": response}

@trace_function(enable_print=False)
def should_continue(state: State):    
    messages = state["messages"]
    if len(messages) >= LIMIT_LENGTH:
        print(f"{YELLOW}\n대화 길이가 {LIMIT_LENGTH} 이상 ({LIMIT_LENGTH//2}번 이상 대화 주고받음) 이므로 요약 노드로 이동합니다.\n{RESET}")
        return "summarize_conversation"
    return END

@trace_function(enable_print=False)
def summarize_conversation(state: State):
    summary = state.get("summary", "")
    if summary:
        summary_message = (
            f"다음은 지금까지의 대화 요약입니다. {summary}\n\n"
            "새로운 메시지를 고려하여 요약 내용을 업데이트하세요."
        )
    else:
        summary_message = "위 내용을 요약하세요."
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:LIMIT_LENGTH//2]]
    return {"summary": response.content, "messages": delete_messages}

workflow = StateGraph(State)
workflow.add_node("conversation", conversation)
workflow.add_node("summarize_conversation", summarize_conversation)
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

In [4]:
config = {"configurable": {"thread_id": "first_chat", 
                           "user_id": "changwoo"}}

In [ ]:
graph.invoke({"messages":"안녕 나는 창우라고해"}, config=config)

In [ ]:
graph.invoke({"messages":"나는 올해 30살이고, 현재 부산대학교에서 박사과정을 하고있어."}, config=config)

In [ ]:
graph.invoke({"messages":"내 전공은 인공지능이고, 요즘 LLM 분야에 관심이 많아."}, config=config)

In [ ]:
graph.invoke({"messages":"내 취미는 헬스장에서 운동하는거야"}, config=config)

In [ ]:
graph.invoke({"messages":"내가 제일 좋아하는 음식은 돼지고기고, 싫어하는 음식은 딱히 없어."}, config=config)

In [ ]:
graph.invoke({"messages":"내 가족관계는 아버지, 어머니, 형, 형수님, 조카 2명이야."}, config=config)

In [ ]:
graph.invoke({"messages":"내 키는 179이고 체중은 84kg이야."}, config=config)